In [3]:
import pandas as pd
import numpy as np
import os
from pymongo import MongoClient

In [5]:
#Path of data files
pospath = r'C:\Anaconda\Galvanize\Minimester2-Project\data\txt_sentoken\pos'
negpath = r'C:\Anaconda\Galvanize\Minimester2-Project\data\txt_sentoken\neg'

#Other initializations including MongoDB and lists of data
client = MongoClient()
db = client.reviews
collection = db.movies
review = []
opinion = []

#Drop MongoDB data so duplicates aren't loaded
collection.drop()

#loop through neg files
for name in os.listdir(negpath):
    #context manager so files aren't floating around
    with open(negpath+'\\'+name, 'r') as fo:
        #read in the data
        data = fo.read()
        #remove apostrophes so that contractions are treated as one word
        #otherwise wouldn't, can't, won't etc will all tokenize to t
        review.append(data.replace("'",""))
        opinion.append(False)

#loop through pos files
for name in os.listdir(pospath):
    #context manager so files aren't floating around
    with open(pospath+'\\'+name, 'r') as fo:
        #read in the data
        data = fo.read()
        review.append(data.replace("'",""))
        opinion.append(True)

df = pd.DataFrame(zip(review,opinion))
df.columns = ['Review','Opinion']
df.head()
result = collection.insert(df.T.to_dict().values())
print collection.count()

2000


In [6]:
sentimentpath = 'C:\Anaconda\Galvanize\Minimester2-Project\data\sentiment'
#Path of data files
pospath, negpath = sentimentpath+r'\positive-words.txt', sentimentpath+r'\negative-words.txt'
#PyMongo variables
client = MongoClient()
db = client['sentiment']
collection = db['bingliu']
#Data lists
words, sentiment, pos, neg = [],[],[],[]
#Drop the existing MongoDB data so duplicates aren't loaded
collection.drop()
#loop through neg files
for name in [pospath, negpath]:
    #context manager so files aren't floating around
    with open(name, 'r') as fo:
        #read in the data
        data = fo.read()
        u_data = data.decode('cp1252') 
        utf8_data = u_data.encode("utf8").split('\n')        
        #Label!
        if name == pospath:
            pos.extend(utf8_data)
        if name == negpath:
            neg.extend(utf8_data)

for word in list(set(pos) & set(neg)):
    #de-dupe words in both pos and neg lexicon
    del pos[pos.index(word)]
    del neg[neg.index(word)]

#now we build the proper sentiment and word lists to load
sentiment.extend([1]*len(pos))
sentiment.extend([-1]*len(neg))
words.extend(pos)
words.extend(neg)

df = pd.DataFrame(zip(words,sentiment))
df.columns = ['Word','Sentiment']
result = collection.insert(df.T.to_dict().values())
print "{} Records loaded".format(collection.count())

6784 Records loaded


In [14]:
#Path of data file
path = 'C:\Anaconda\Galvanize\Minimester2-Project\data\sentiment\subjclueslen1-HLTEMNLP05.tff'
#PyMongo variables
client = MongoClient()
db = client['sentiment']
collection = db['mpqa']
collection.drop()
sentiment_df = pd.read_csv(path, sep=' ')
#for some reason I'm unable to read the Stemmed1 column with that title
#so I'm using S here
sentiment_df.columns = ['Type','Len', 'Word1', 'Pos1','S',\
                        'PriorPolarity']
sentiment_df.iloc[:, 0] = sentiment_df.iloc[:, 0].str.replace('type=', '')
sentiment_df.iloc[:, 1] = sentiment_df.iloc[:, 1].str.replace('len=', '')
sentiment_df.iloc[:, 2] = sentiment_df.iloc[:, 2].str.replace('word1=', '')
sentiment_df.iloc[:, 3] = sentiment_df.iloc[:, 3].str.replace('pos1=', '')
sentiment_df.iloc[:, 4] = sentiment_df.iloc[:, 4].str.replace('stemmed1=', '')
sentiment_df.iloc[:, 5] = sentiment_df.iloc[:, 5].str.replace('priorpolarity=', '')

result = collection.insert(sentiment_df.T.to_dict().values())
print "{} Records loaded".format(collection.count())

sentiment_df.head()

8221 Records loaded


,Type,Len,Word1,Pos1,S,PriorPolarity
0,weaksubj,1,abandonment,noun,n,negative
1,weaksubj,1,abandon,verb,y,negative
2,strongsubj,1,abase,verb,y,negative
3,strongsubj,1,abasement,anypos,y,negative
4,strongsubj,1,abash,verb,y,negative
